In [1]:
%matplotlib inline

import os
import numpy as np
import tensorflow as tf
import data_loader
import config
import audio_processing

from tqdm import tqdm

In [29]:
conf = config.get_config()
f = "warm_vs_metallic"
conf.update({
    "features": [f],
    "valid_split": 0.0
})

train, valid = data_loader.data_loader(conf)

2021-05-06 13:38:07.007 | INFO     | data_loader:data_loader:28 - Loading csv and checking audio files
2021-05-06 13:38:07.016 | INFO     | data_loader:data_loader:31 - Creating dataset
2021-05-06 13:38:07.017 | INFO     | data_loader:data_loader:34 - Validation split is 0.0
2021-05-06 13:38:07.142 | INFO     | data_loader:data_loader:47 - Creating train and valid splits


In [30]:
min_level = 50 - conf.get("threshold")
max_level = 50 + conf.get("threshold")

num_examples = len(train)

x_train = np.zeros((num_examples, 513, 3))
y_train = np.zeros((num_examples, ))

for i, (key, value) in tqdm(enumerate(train.items())):
    current_class = 1
    file_path = os.path.join(conf.get("base_dir"), f"{key}.wav")

    for _, feature in enumerate(conf.get("features")):
        current_val = int(value[feature])
        if current_val < min_level:
            current_class = 0
        elif current_val > max_level:
            current_class = 2

    y_train[i] = current_class
    h, p, r = audio_processing.get_hpr(file_path, conf)
    h = np.abs(h).mean(axis=-1)
    p = np.abs(p).mean(axis=-1)
    r = np.abs(r).mean(axis=-1)
    x_train[i, :, 0] = h
    x_train[i, :, 1] = p
    x_train[i, :, 2] = r

1569it [01:20, 19.51it/s]


In [31]:
x_train = x_train / x_train.max()

In [32]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(513, 3)),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(3, activation="softmax")
])

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 1539)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 512)               788480    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 64)                32832     
_________________________________________________________________
dropout_9 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 3)                 195       
Total params: 821,507
Trainable params: 821,507
Non-trainable params: 0
________________________________________________

In [34]:
_ = model.fit(
    x_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=16,
    verbose=2
)

Epoch 1/50
79/79 - 0s - loss: 0.8935 - accuracy: 0.5426 - val_loss: 1.0960 - val_accuracy: 0.4841
Epoch 2/50
79/79 - 0s - loss: 0.8839 - accuracy: 0.5514 - val_loss: 1.0914 - val_accuracy: 0.4713
Epoch 3/50
79/79 - 0s - loss: 0.8950 - accuracy: 0.5618 - val_loss: 1.0997 - val_accuracy: 0.4777
Epoch 4/50
79/79 - 0s - loss: 0.8755 - accuracy: 0.5538 - val_loss: 1.1304 - val_accuracy: 0.5096
Epoch 5/50
79/79 - 0s - loss: 0.8824 - accuracy: 0.5378 - val_loss: 1.1251 - val_accuracy: 0.4459
Epoch 6/50
79/79 - 0s - loss: 0.8836 - accuracy: 0.5546 - val_loss: 1.1214 - val_accuracy: 0.4618
Epoch 7/50
79/79 - 0s - loss: 0.8737 - accuracy: 0.5618 - val_loss: 1.1233 - val_accuracy: 0.4968
Epoch 8/50
79/79 - 0s - loss: 0.8733 - accuracy: 0.5705 - val_loss: 1.1340 - val_accuracy: 0.4554
Epoch 9/50
79/79 - 0s - loss: 0.8718 - accuracy: 0.5434 - val_loss: 1.1452 - val_accuracy: 0.4777
Epoch 10/50
79/79 - 0s - loss: 0.8653 - accuracy: 0.5753 - val_loss: 1.1377 - val_accuracy: 0.4809
Epoch 11/50
79/79 -